# Angular acceptance of the decay products

## Import ROOT and LHEF libraries

In [2]:
import csv
import os
import random
import sys
import numpy as np
import ROOT as r
import matplotlib.pyplot as plt
r.gROOT.ProcessLine(".include /Users/isaac/Work/MG5_aMC_v3_1_0")
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v3_1_0/ExRootAnalysis/\
ExRootAnalysis/ExRootTreeReader.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v3_1_0/\
ExRootAnalysis/ExRootAnalysis/ExRootClasses.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v3_1_0/ExRootAnalysis/\
ExRootAnalysis/ExRootLHEFReader.h"'
)
r.gSystem.Load(
    "/Users/isaac/Work/MG5_aMC_v3_1_0/\
ExRootAnalysis/libExRootAnalysis.so"
)
os.environ["TERM"] = "linux"
random.seed(1)


Welcome to JupyROOT 6.22/09


## Extract dark photon data

Here I extract the energy, momentum and $\theta$ of the dark photon. Given that reading rootfile is slow, I'll store the data list here for further use. After getting the data list, rootfile is no longer needed.

In [9]:
chain = r.TChain("LHEF")
chain.Add("~/Work/MUonE/MG5_events/100mev/Events/run_01/unweighted_events.root")
X_list = []
for event in chain:
    for particle in event.Particle:
        if particle.PID == 103:
            E_X = particle.E
            p_X = np.sqrt(particle.E**2 - particle.M**2)
            X_vector = r.TLorentzVector()
            X_vector.SetPtEtaPhiM(particle.PT, particle.Eta, particle.Phi, particle.M)
            theta_X = X_vector.Theta()
    X_list.append([E_X, p_X, theta_X])

## Generate the angle of decay products

In the rest frame, the decay product should have $\phi$ and $\cos \theta$ is uniformly distributed. We can generate the number simply by taking use of the random number generator.

Further, the two final state particles must distribute on the different "side" of the dark photon. Let's put the dark photon at $x=0$, $y>0$ plane while define the main axis is $x=y=0$. The dark photon is called to be "going up". One of the final state electron/positron must go further up than the dark photon, while the other must below the dark photon (the latter one may still be "up" compared to the main axis, but must be "down" compared to the dark photon). It's always easier for the upper one to escape from the angular acceptance. Thus, we can just focus on the upper one.

We define the upper one to have the $\theta$ with the dark photon trajectory, and rotate $\phi$ angle with respect to the dark photon trajectory (regard the trajectory as axis!). $\phi=0$ means that this particle to be along the $yz$ plane. This is "upper one", so this angle vary from $-\frac{\pi}{2}$ to $\frac{\pi}{2}$. $\theta$ should vary from $0$ to $\pi$, where $0$ means the same direction along the dark photon.

Boost changes the $\theta$ into the lab frame while leaving $\phi$ unchanged. Note that this $\phi$ is not the angle to rotate along z-axis! But since $\theta_A$ and $\theta$ are both small, we can take the approximation that the $\phi$ w.r.t dark photon axis equals to that w.r.t z-axis.

## Generate the decay length

The decay length of the dark photon is given by (eq 11)
$$
d_{A^\prime}=\frac{p_{A^\prime}}{m_{A^\prime} \Gamma_{A^\prime}}
$$
where the $p$ is momentum, $m$ is mass and $\Gamma$ is the decay rate (eq 10).

The differential probability for the dark photon to decay at distance $(x, x+dx)$ is
$$
\frac{dP(x)}{dx} = \frac{1}{d_{A^\prime}} \exp{\left (-\frac{x}{d_{A^\prime}}\right)}
$$

## Geometry on the detector plane

![Geometry](./geometry.png)

Suppose that is a circle with radius $r$. As shown in the image, the dark photon, if not decay, will hit the point B. This dark photon actually decay at point A, and the upper product hits point C on the detector. The angle $\phi$ in the image, is the angle between $BC$ and the vertical radius. This is very close to the azimuthal angle in the dark photon rest frame when calculate the decay process, given that the $\theta_e$ is very small.

The distances are got from the following:
- $AB$: this is similar to $AC$, equals to the total distance $OB \simeq 1 \rm m$ minus the dark photon decay distance $OA$.
- $BC$: Given by $AB \times \theta_e$.
- $BD$: the vertical distance, given as $OD \times \theta_A$.
- Upper limit of $BC$: this should be the distance from $B$ to the edge of the circle along $BC$, playing the role as the upper bound for the angular acceptance. This is given by
  $$
    \frac{\sin (\phi - \alpha)}{\sin(\phi)}r
  $$
  where $\sin \alpha = \frac{BD}{r}\sin \phi$

## Boost of the particle

We focus on the "upper" product. Define all the angles as discussed above. The $\theta_{elab}$ in the lab frame should satisfy
$$
\cos \theta_{elab} = \frac{\gamma (p_{com} \cos \theta_e + \beta \frac{1}{2}m_{A^\prime})}{\sqrt{p_{com}^2 \sin^2 \theta_e + \gamma^2 (p_{com} \cos \theta + \beta \frac{1}{2}m_{A^\prime})^2}}
$$
where the $\gamma = E_{A^\prime}/m_{A^\prime}$ and $\beta=p_{A^\prime}/E_{A^\prime}$ are the boost factors. The $p_{com} = \frac{1}{2}\sqrt{m_{A^\prime}^2 - 4m_e^2}$ is the 3-momentum of the decay product in the rest frame. $\theta_e$ is nothing but the $\theta$ angle of the decay product in the rest frame.

In [3]:
# Basic quantites and functions
luminosity = 1.5e04
m_e = 5.11e-04
meter = 1.0
GeV = 1 / (1.97 * meter * 1.0e-16)
Radius = 0.05

angle_cut = 1.0 / 1000.0  # input mrad, change to rad
L_min = 0.02
L_max = 14.5 / 100.0  # input cm, change to m

def decay_rate(mass, g_e):
    # the \Gamma, eq 10
    alpha_e = (g_e ** 2) / (4 * np.pi)
    return (
        alpha_e
        * mass
        * (1 + 2 * ((m_e ** 2) / (mass ** 2)))
        * np.sqrt(1 - 4 * ((m_e ** 2) / (mass ** 2)))
        * GeV
        / 3
    )


def distance(momentum, mass, g_e):
    # the d_{A^\prime}, eq 11
    return momentum / (decay_rate(mass, g_e) * mass)

def boost(theta, mass, energy):
    beta = np.sqrt(energy**2.0 - mass**2.0)/energy
    gamma = energy/mass
    pcom = np.sqrt(mass**2.0 - 4.0 * (m_e ** 2.0))/2.0
    nominator = gamma * (pcom * np.cos(theta) + beta * mass / 2.0)
    denominator = np.sqrt((pcom**2.0) * (np.sin(theta)**2.0) + (gamma**2.0)*((pcom * np.cos(theta) + beta * mass * 0.5)**2.0))
    return nominator/denominator

In [77]:
# Angular acceptance in the trackers (not in the ECAL)
good_photon_number = 0
good_decay_number = 0
for i in range(10000):
    E_X = X_list[i][0]
    p_X = X_list[i][1]
    theta_X = X_list[i][2]
    # generate decay distance
    d_A = np.random.exponential(scale=distance(p_X, 0.002, 1.0e-04)) # take an example of g_e=1e-4
    # Add angular acceptance cut on the dark photon. At least the dark photon itself should be inside the angular acceptance.
    if(1.0 * theta_X <= Radius and L_min <= d_A <= L_max):
        good_photon_number += 1
        # generate decay angle
        theta_e = np.arccos(np.random.uniform(-1.0, 1.0))
        phi_e = np.random.uniform(- np.pi*0.5, np.pi*0.5)
        theta_elab = np.arccos(boost(theta_e, 0.002, E_X))
        AB = 1.0 - d_A
        BC = AB * theta_elab
        BD = 1.0 * theta_X
        sin_alpha = np.sin(phi_e) * BD/Radius
        alpha = np.arcsin(sin_alpha)
        BCmax = Radius * (np.sin(phi_e - alpha)/np.sin(phi_e))
        if BC <= BCmax:
            good_decay_number += 1


print(good_photon_number) # This is the number normalized with 10000 events
print(good_decay_number)

166
166


Now let's renormalize according to the xsec and luminosity

This number agrees with the result which I used to generate the reach plot, even though here we added an angular cut on dark photon so that the dark photon itself will point inside the last tracker of the module.

And we can see above that for this benchmark point, the opening angle did not change anything if we require the dark photon to be inside the angular acceptance.

Now this code already investigated how we apply the geometrical cuts. The following problem is simply what is the best way to show the numbers. 

My suggestion: compare the number of events that enters the angular accpetance, and that we only require the dark photon to decay in volume. Show how this probability changes with mass.

(If we already do such detailed calculation, why trouble ourself by proving that angular acceptance is garanteed for large mass range? Why not directly apply this cut on the final reach plot?)

In [4]:
chain = r.TChain("LHEF")
chain.Add("../../MG5_events/2to3vec/Events/X_mass_0.2057/unweighted_events.root")

1

In [6]:
X_list = []
for event in chain:
    for particle in event.Particle:
        if particle.PID == 103:
            E_X = particle.E
            p_X = np.sqrt(particle.E**2 - particle.M**2)
            X_vector = r.TLorentzVector()
            X_vector.SetPtEtaPhiM(particle.PT, particle.Eta, particle.Phi, particle.M)
            theta_X = X_vector.Theta()
    X_list.append([E_X, p_X, theta_X])

In [8]:
# Angular acceptance in the trackers (not in the ECAL)
good_photon_number = 0
good_decay_number = 0
for i in range(len(X_list)):
    E_X = X_list[i][0]
    p_X = X_list[i][1]
    theta_X = X_list[i][2]
    # generate decay distance
    d_A = np.random.exponential(scale=distance(p_X, 0.2057, 1.0e-04)) # take an example of g_e=1e-4
    # Add angular acceptance cut on the dark photon. At least the dark photon itself should be inside the angular acceptance.
    if(L_max * theta_X and L_min <= d_A <= L_max):
        good_photon_number += 1
        # generate decay angle
        theta_e = np.arccos(np.random.uniform(-1.0, 1.0))
        phi_e = np.random.uniform(- np.pi*0.5, np.pi*0.5)
        theta_elab = np.arccos(boost(theta_e, 0.2057, E_X))
        AB = 1.0 - d_A
        BC = AB * theta_elab
        BD = 1.0 * theta_X
        if BD <= Radius:
            sin_alpha = np.sin(phi_e) * BD/Radius
            alpha = np.arcsin(sin_alpha)
            BCmax = Radius * (np.sin(phi_e - alpha)/np.sin(phi_e))
            if BC <= BCmax:
                good_decay_number += 1

efficiency_rate = float(good_decay_number)/float(good_photon_number)
print(efficiency_rate)
print(good_decay_number)

1.0
5


In [7]:
good_photon_number = 0
good_decay_number = 0
for j in range(0, 40):
    axis = j+1 # The distance from the j-th target to the ECAL. j is counted from the one that is closest to the ECAL
    for i in range(0, len(X_list)):
        E_X = X_list[i][0]
        p_X = X_list[i][1]
        theta_X = X_list[i][2]
        d_A = np.random.exponential(scale=distance(p_X, 0.2057, 1.0e-04))
        if (L_max * theta_X <= Radius and L_min <= d_A <= L_max):
            good_photon_number += 1
            theta_e = np.arccos(np.random.uniform(-1.0, 1.0))
            phi_e = np.random.uniform(-np.pi*0.5, np.pi*0.5)
            theta_elab = np.arccos(boost(theta_e, 0.2057, E_X))
            AB = axis - d_A
            BC = AB*theta_elab
            BD = axis*theta_X
            if BD <= Radius:
                sin_alpha = np.sin(phi_e) * BD/Radius
                alpha = np.arcsin(sin_alpha)
                BCmax = Radius * (np.sin(phi_e - alpha)/np.sin(phi_e))
                if BC <= BCmax: good_decay_number += 1
good_photon_number = good_photon_number/40
good_decay_number = good_decay_number/40
efficiency_rate = float(good_decay_number)/float(good_photon_number)
print(good_photon_number)
print(good_decay_number)
print(efficiency_rate)

3.3
2.025
0.6136363636363636


In [13]:
coupling_list = [10.0 ** power for power in np.arange(-6.0, -2.0, 0.2)]
print(coupling_list)

[1e-06, 1.584893192461114e-06, 2.5118864315095823e-06, 3.981071705534978e-06, 6.309573444801943e-06, 1.0000000000000021e-05, 1.5848931924611175e-05, 2.5118864315095873e-05, 3.981071705534986e-05, 6.309573444801955e-05, 0.00010000000000000041, 0.00015848931924611207, 0.00025118864315095926, 0.00039810717055349936, 0.0006309573444801969, 0.001000000000000006, 0.0015848931924611238, 0.0025118864315095977, 0.003981071705535002, 0.0063095734448019814]


In [12]:
print(np.logspace(-6.0, -2.0, 20).tolist())

[1e-06, 1.6237767391887209e-06, 2.6366508987303555e-06, 4.281332398719396e-06, 6.951927961775606e-06, 1.1288378916846883e-05, 1.8329807108324375e-05, 2.9763514416313192e-05, 4.8329302385717524e-05, 7.847599703514606e-05, 0.00012742749857031334, 0.00020691380811147902, 0.0003359818286283781, 0.0005455594781168515, 0.0008858667904100823, 0.0014384498882876629, 0.002335721469090121, 0.003792690190732246, 0.00615848211066026, 0.01]
